<a href="https://colab.research.google.com/github/JacobHSampson/FranklinU/blob/main/Copy_of_Assignment_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment for Week 7

## Purpose          
This assignment is intended to help you do the following:      
  - Groupby operations to aggregate, transform, and filter data      
  - Built-in and custom user functions to perform groupby operations


## About the assignment

The function 'get_data_table' takes 4 parameters, df, date_col, group_col, type_x.      
  - The 'df' represents the dataframe.      
  -The 'date_col' represents the date column.      
  -The group_col represents the column to groupby and        
  -The 'type_x' represents the column with a specific value/category == 'type_x'. 

In [3]:
def get_data_table(df, date_col, group_col, type_x):
    df_gross = (df.loc[(df.type == type_x) &
                                      df.category.isin(['category_x', 'category_xy', 'category_xyz'])
                                     ]
                         .groupby([group_col, date_col]).sum()
                         .unstack(date_col)['amount'].fillna(0)
                         .resample('M', axis=1).sum()
                        )

    df_recovered = (df.loc[(df.type == type_x) &
                                  df.category.isin(['category_01', 'category_05', 'category_07'])
                                 ]
                    .groupby([group_col, date_col]).sum()
                    .unstack(date_col)['amount'].fillna(0)
                    .resample('M', axis=1).sum()
                    )

    return df_gross.add(df_recovered * -1, fill_value=0)
    

## Tasks   
  1. Read the function above and in a 1-3 paragraphs explain its purpose    

  2. Apply the function to your selected dataframe and modify as needed based on your preferences.    

  3. Write 1-3 paragraphs to explain the differences or challenges when you applied the function to your chosen dataframe.

  4. Post your version of the above code on your GitHub account. Do not forget a README file with explanations.




**1)**
 
  The purpose of the function is to create two 'pivoted' categories (gross and recovered), and at the end recovered is subtracted from gross. Both gross and recovered use the rows that match the values specified in type_x and in a certain category.
  These values are grouped by the groupby column and the date column (and summed up), and then the date column is unstacked (ungrouped) by a certain value (what is in the ['amount'] part. Missing values are filled with zero. The dates are then resampled by month and the value for amount are summed up.

**2)**


In [22]:
import pandas as pd
import numpy as np
df = pd.read_csv('https://raw.githubusercontent.com/frankData612/data_612/master/stock_data/stocks_yahoo.csv',parse_dates =['date'])

df.dtypes


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,no,date,company_name,price_at_close,price_change,price_after_hours,previous_close,today opened,price range,52 week range,volume,average volume,market cap,beta (3Y Monthly),PE Ratio,EPS,earnings date,Dividend Yield,1Y target est
0,1,2019-11-21,GOOG - Alphabet Inc.,1301.35,-1.70 (-0.13%),1301.35,1303.05,1301.48,"1,293.01 - 1,312.45","970.11 - 1,335.53",825205,1280155.0,897.113B,1.01,27.93,46.60,NaN,N/A (N/A),1487.52
56,57,2019-11-22,GOOG - Alphabet Inc.,1295.34,-6.01 (-0.46%),1295.34,1301.35,1305.62,"1,291.41 - 1,308.73","970.11 - 1,335.53",1.38651e+06,1284626.0,892.963B,1.01,27.80,46.60,NaN,N/A (N/A),1487.52
260,261,2019-11-25,GOOG - Alphabet Inc.,1306.69,+11.35 (+0.88%),1306.69,1295.34,1299.18,"1,298.14 - 1,311.31","970.11 - 1,335.53",1.03649e+06,1284626.0,900.624B,1.01,28.04,46.60,NaN,N/A (N/A),1487.52
614,615,2019-12-04,GOOG - Alphabet Inc.,1320.54,+25.26 (+1.95%),1320.54,1295.28,1307.01,"1,304.98 - 1,325.80","970.11 - 1,335.53",1.489e+06,1277435.0,910.569B,1.01,28.34,46.60,NaN,N/A (N/A),1501.98
759,760,2019-12-05,GOOG - Alphabet Inc.,1328.13,+7.59 (+0.57%),1328.13,1320.54,1328.00,"1,316.63 - 1,329.07","970.11 - 1,335.53",1.01201e+06,1284757.0,915.437B,1.02,28.50,46.60,NaN,N/A (N/A),1501.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64142,64143,2020-04-22,GOOG - Alphabet Inc.,1263.21,+46.87 (+3.85%),1263.21,1216.34,1245.54,"1,242.07 - 1,285.61","1,013.54 - 1,532.11",2093140.00,2403628.0,866.683B,1.05,25.69,49.16,NaN,N/A (N/A),1497.90
64508,64509,2020-04-23,GOOG - Alphabet Inc.,1276.31,+13.10 (+1.04%),1276.31,1263.21,1271.55,"1,265.67 - 1,292.96","1,013.54 - 1,532.11",1196906.00,2411273.0,875.51B,1.05,25.96,49.16,NaN,N/A (N/A),1497.90
65147,65148,2020-04-24,GOOG - Alphabet Inc.,1279.31,+3.00 (+0.24%),1279.31,1276.31,1261.17,"1,249.83 - 1,280.40","1,013.54 - 1,532.11",1640394.00,2414696.0,877.467B,1.05,26.02,49.16,NaN,N/A (N/A),1497.90
66073,66074,2020-04-27,GOOG - Alphabet Inc.,1275.88,-3.43 (-0.27%),1275.88,1279.31,1296.00,"1,269.00 - 1,296.15","1,013.54 - 1,532.11",1600563.00,2412395.0,875.959B,1.05,25.95,49.16,NaN,N/A (N/A),1497.90


In [13]:
df_price_pc = (df.loc[(df['company_name'] == 'GOOG - Alphabet Inc.')]).groupby(['date','company_name']).sum().unstack('date')['price_at_close'].fillna(0).resample('Q', axis=1).sum() 
df_est = (df.loc[(df['company_name'] == 'GOOG - Alphabet Inc.')]).groupby(['date','company_name']).sum().unstack('date')['1Y target est'].fillna(0).resample('Q', axis=1).sum() 
df_est.add(df_price_pc * -1, fill_value=0)

date,2019-12-31,2020-03-31,2020-06-30
company_name,,,
GOOG - Alphabet Inc.,3357.94,12970.23,5854.13


In [25]:
def get_data_table(df,date_col,group_col,type_x):
    df_price_pc = (df.loc[(df['company_name'] == 'GOOG - Alphabet Inc.')]).groupby(['date','company_name']).sum().unstack('date')['price_at_close'].fillna(0).resample('W', axis=1).mean() 
    df_est = (df.loc[(df['company_name'] == 'GOOG - Alphabet Inc.')]).groupby(['date','company_name']).sum().unstack('date')['1Y target est'].fillna(0).resample('W', axis=1).mean() 
    return df_est.add(df_price_pc * -1, fill_value=0)

In [26]:
get_data_table(df, 'date', 'company_name', 'GOOG - Alphabet Inc.')

date,2019-11-24,2019-12-01,2019-12-08,2019-12-15,2019-12-22,2019-12-29,2020-01-05,2020-01-12,2020-01-19,2020-01-26,2020-02-02,2020-02-09,2020-02-16,2020-02-23,2020-03-01,2020-03-08,2020-03-15,2020-03-22,2020-03-29,2020-04-05,2020-04-12,2020-04-19,2020-04-26,2020-05-03
company_name,,,,,,,,,,,,,,,,,,,,,,,,
GOOG - Alphabet Inc.,189.175,180.83,172.216667,155.712,147.072,150.8075,146.963333,115.53,54.155,57.165,104.095,122.134,100.7825,103.965,244.242,269.476,420.24,528.656,494.316,431.418,347.96,279.094,248.316,243.125


**3)**

One of the main differences in my function are omitting the df.category.isin(['category_x', 'category_xy', 'category_xyz']) part for each of the two categories, which in my function are price_after_close and 1Y target estimate. Another difference is I resample based on the week and take the mean instead of the sum.

I do not think the original function means for the group_col to be the same in the df.type = type_x. The challenge for the data set I chose is there are not many columns to group by. That is why chose not to have the df.category.isin(['category_x', 'category_xy', 'category_xyz']) part in my function. I could have used that part instead of df.type = type_x and included multiple companies to compare.

I chose to use price_at_close and 1Y target est to compare how far a given company is from their target any given week in the year.